requiremenrs.txt

langchain
langchain-openai
langchainhub # langchain python라이브러리로 프롬프트, 에이전트, 체인 관련 패키지 모음
langserve[all]

faiss-cpu  # Facebook에서 개발 및 배포한 밀집 벡터의 유사도 측정, 클러스터링에 효율적인 라이브러리
tavily-python # 언어 모델에 중립적인 디자인으로, 모든 LLM과 통합이 가능하도록 설계된 검색 API
beautifulsoup4  #파이썬에서 사용할 수 있는 웹데이터 크롤링 라이브러리
wikipedia

fastapi #  Python의 API를 빌드하기 위한 웹 프레임워크
uvicorn # ASGI(Asynchronous Server Gateway Interface) 서버
urllib3 # 파이썬에서 HTTP 요청을 보내고 받는 데 사용되는 강력하고 유연한 라이브러리

python-dotenv
pypdf

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install python-dotenv
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu
!pip install wikipedia
!pip install openai

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아 등록해야 함.

[Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록

In [ ]:
import os

# TAVILY API KEY를 기입합니다.
os.environ["TAVILY_API_KEY"] = "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"

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"

In [ ]:
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
#google drive load
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# training_dir_path = '/content/drive/MyDrive/2025_Bigdata_nlp_class/aihub_dataset/Training/02_label_data'
# validation_dir_path = '/content/drive/MyDrive/2025_Bigdata_nlp_class/aihub_dataset/Validation/02_label_data'

search.invoke 함수는 주어진 문자열에 대한 검색을 실행

invoke() 함수에 검색하고 싶은 검색어를 넣어 검색을 수행

#PDF 기반 문서 검색 도구: Retriever

내부 데이터에 대해 조회를 수행할 retriever 생성.

*   웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축
*   PDF 문서를 FAISS DB 에 저장하고 조회하는 retriever 를 생성



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
import json
from langchain.document_loaders import TextLoader
from langchain.schema import Document
import unicodedata # Import the unicodedata module

# PDF 파일 로드. 파일의 경로 입력
# 경로 설정
training_dir_path = "/content/drive/MyDrive/2025_Bigdata_nlp_class/aihub_dataset/Training/02_label_data"

# for f in os.listdir(training_dir_path):
#     print(repr(f))  # unicode escape로 특수문자 확인

documents = []
# === 1. JSON 파일 로드 + 파일명 정규화 → 메타데이터 추출 ===
def load_documents_with_metadata(folder_path):
    for raw_filename in os.listdir(folder_path):
        # 파일 시스템에서 읽은 원래 이름을 정규화(NFC) 처리
        filename = unicodedata.normalize("NFC", raw_filename)
        file_path = os.path.join(folder_path, raw_filename)  # 실제 파일 경로는 raw_filename 써야 합니다.
        print(f"▶ 처리 중 파일 (원본): {filename}")

        # 실제 파일인지 확인 (폴더나 시스템 파일 스킵)
        if not os.path.isfile(file_path):
            continue
        # .json 확장자가 아닌 파일 스킵
        if not filename.endswith(".json"):
            continue

        try:
            # 정규화된 파일명 출력 (한글 조합형 → 완성형으로 변환됐는지 확인)
            print(f"▶ 처리 중 파일 (정규화): {filename}")

            # 정규화된 파일명을 "_"로 분리
            # 예시: Empathy_기쁨_부모자녀_조손_343.json
            parts = filename.replace(".json", "").split("_")

            # parts[1] = 감정, parts[2] = 관계  (예: "기쁨", "부모자녀")
            if len(parts) >= 3:
                emotion = parts[1]
                relation = parts[2]
            else:
                emotion = "unknown"
                relation = "unknown"

            # JSON 읽기
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                utterances = data.get("utterances", [])

                # 대화 utterance만 합쳐서 하나의 긴 텍스트로 만듦
                full_text = "\n".join([utt.get("text", "") for utt in utterances])

                # 텍스트가 비어 있으면 스킵
                if full_text.strip() == "":
                    print(f"  ⚠️ 내용 비어 있음 → 스킵: {filename}")
                    continue

                # Document 생성
                doc = Document(
                    page_content=full_text,
                    metadata={
                        "filename": filename,
                        "emotion": emotion,
                        "relation": relation
                    }
                )
                documents.append(doc)

        except Exception as e:
            print(f"❌ 오류 발생 ({filename}): {e}")

    return documents


documents = load_documents_with_metadata(training_dir_path)
print(f"✅ 로드된 원본 문서 수: {len(documents)}")

In [ ]:
# === 2. 문서 분할 함수 ===
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    return splitter.split_documents(documents)


# === 3. FAISS DB 생성 ===
def create_faiss_db(docs):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

def filtered_similarity_search(vectorstore, query, emotion=None, relation=None, k=3):
    # docstore 내 모든 Document 객체 가져오기
    all_docs = list(vectorstore.docstore._dict.values())

    # 1차 감정(emotion), 2차 관계(relation) 필터링
    filtered_docs = []
    for doc in all_docs:
        doc_em = doc.metadata.get("emotion", "")
        doc_rel = doc.metadata.get("relation", "")
        if emotion and doc_em != emotion:
            continue
        if relation and doc_rel != relation:
            continue
        filtered_docs.append(doc)

    if not filtered_docs:
        print("❗ 해당 감정/관계 조건의 문서가 없습니다.")
        return []

    # 필터링된 문서를 임베딩해서 별도의 FAISS 인덱스로 만들 수도 있지만,
    # 여기서는 간단히 vectorstore.similarity_search() 호출 → 결과에서 필터 적용
    # (단, 필요하다면 filtered_docs만으로 새로운 FAISS 인덱스를 생성 후 검색할 수도 있습니다.)
    results = vectorstore.similarity_search(query, k=k)

    # 검색 결과 중에서도 meta 필터(감정/관계)가 맞는 것만 리턴
    return [doc for doc in results
            if (not emotion or doc.metadata.get("emotion") == emotion)
            and (not relation or doc.metadata.get("relation") == relation)]

In [ ]:
# 2) 문서 분할 (chunking)
split_docs = split_documents(documents)
print(f"✅ 분할된 문서 청크 개수: {len(split_docs)}")

In [ ]:
# 예시: split_docs 리스트에서 앞의 10개 문서만 확인하는 코드

# (이전 단계에서 이미 split_docs를 생성했다고 가정)
# split_docs = split_documents(documents)

# 앞 10개 문서만 출력
for idx, doc in enumerate(split_docs[:5], start=1):
    print(f"--- 문서 #{idx} ---")
    print(f"파일명    : {doc.metadata.get('filename')}")
    print(f"감정      : {doc.metadata.get('emotion')}")
    print(f"관계      : {doc.metadata.get('relation')}")
    print("내용 (일부) :")
    print(doc.page_content[:200].replace("\n", " ") + "...\n")

In [ ]:
# 3) FAISS DB 생성
if not split_docs:
  raise RuntimeError("❌ 분할된 문서가 없어서 FAISS 생성이 불가능합니다.")
faiss_db = create_faiss_db(split_docs)
print("✅ FAISS DB 생성 완료")

In [ ]:
# 저장/로딩할 FAISS 인덱스 폴더 경로
index_dir = "/content/drive/MyDrive/2025_Bigdata_nlp_class/faiss_index"

# --- (1) 이미 저장된 인덱스가 있는지 확인 ---
if os.path.isdir(index_dir) and \
   os.path.exists(os.path.join(index_dir, "index.faiss")):
    # 저장된 인덱스가 있으면 로드
    embeddings = OpenAIEmbeddings()
    # allow_dangerous_deserialization=True 를 추가하여 로딩을 허용합니다.
    faiss_db = FAISS.load_local(index_dir, embeddings, allow_dangerous_deserialization=True)
    print("✅ 기존 FAISS 인덱스를 불러왔습니다:", index_dir)

else:
    # 저장된 인덱스가 없으면 새로 생성
    # ① 여기에 split_docs를 미리 생성하는 코드를 넣으세요
    #    예시: split_docs = split_documents(documents)
    #
    # ② create_faiss_db 함수나 직접 임베딩 + 저장 로직을 호출
    #
    # 예시:
    # split_docs = split_documents(documents)
    # embeddings = OpenAIEmbeddings()
    # faiss_db = FAISS.from_documents(split_docs, embeddings)
    #
    # 실제 프로젝트에 맞게 아래 두 줄을 수정하세요:
    faiss_db = create_faiss_db(split_docs)
    embeddings = OpenAIEmbeddings()

    # --- (2) 생성된 FAISS를 로컬에 저장 ---
    os.makedirs(index_dir, exist_ok=True)
    faiss_db.save_local(index_dir)
    print("✅ 새로 FAISS 인덱스를 생성하고 저장했습니다:", index_dir)

# 이후 faiss_db를 retriever로 사용 가능합니다.
# 예시:
retriever = faiss_db.as_retriever()

In [ ]:
# === 4. 사용자 검색 함수 (감정 + 관계 필터) ===
def filtered_similarity_search(vectorstore, query, emotion=None, relation=None, k=3):
    # 필터링
    all_docs = vectorstore.docstore._dict.values()
    filtered_docs = [
        doc for doc in all_docs
        if (emotion is None or doc.metadata.get("emotion") == emotion)
        and (relation is None or relation in doc.metadata.get("relation"))
    ]

    if not filtered_docs:
        print("❗해당 조건의 문서가 없습니다.")
        return []

    # 유사도 기반 검색
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    query_chunks = splitter.split_text(query)
    search_results = []
    for chunk in query_chunks:
        search_results.extend(vectorstore.similarity_search(chunk, k=k))
    return search_results

In [ ]:
# === 예시 검색 ===
query = "아기를 키우는 게 너무 힘들어요. 조언이 필요해요."
results = filtered_similarity_search(faiss_db, query, emotion="기쁨", relation="부모자녀")

for i, doc in enumerate(results):
    print(f"\n✅ 검색 결과 {i+1}")
    print(f"파일명: {doc.metadata['filename']}")
    print(f"감정: {doc.metadata['emotion']} / 관계: {doc.metadata['relation']}")
    print(doc.page_content[:300] + "...")

In [ ]:
import openai

# OpenAI API 키 설정
class config:
    OPENAI_API_KEY = ""

openai.api_key = config.OPENAI_API_KEY

# ─── 2) GPT-4o(또는 GPT-4o-mini)를 사용해 “가장 적절한” 문서를 고르는 함수 ───────────────────
def choose_best_doc_with_gpt(query, docs, model="gpt-4o-mini"):
    """
    query: 사용자의 원래 질문
    docs: filtered_similarity_search에서 반환된 Document 객체 리스트
    model: "gpt-4o" 또는 "gpt-4o-mini"
    반환: (best_doc, gpt_explanation)
    """
    # (A) 프롬프트 구성
    prompt_parts = [
        "당신은 대화 응답 후보를 평가하는 전문가입니다.\n",
        f"사용자 질문: \"{query}\"\n",
        "다음은 검색된 응답 후보들입니다.\n"
    ]

    for idx, doc in enumerate(docs, start=1):
        snippet = doc.page_content.strip().replace("\n", " ")
        if len(snippet) > 300:
            snippet = snippet[:300] + "..."
        prompt_parts.append(
            f"[{idx}]\n"
            f"Filename: {doc.metadata.get('filename')}\n"
            f"Emotion: {doc.metadata.get('emotion')}, Relation: {doc.metadata.get('relation')}\n"
            f"Content: \"{snippet}\"\n"
        )

    prompt_parts.append(
        "\n위 후보들 중에서, 사용자 질문에 가장 적절한 응답을 하나 선택하고, 그 이유를 간단히 설명해주세요.\n"
        "반드시 다음 형식으로 응답해야 합니다:\n"
        "선택: [번호]\n"
        "이유: [이유]\n"
    )

    full_prompt = "\n".join(prompt_parts)

    # (B) GPT-4o 호출
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 뛰어난 대화 평가자입니다."},
            {"role": "user", "content": full_prompt}
        ],
        max_tokens=300,
        temperature=0.0
    )

    gpt_reply = response.choices[0].message.content.strip()

    # (C) GPT가 반환한 '선택: [번호]' 파싱
    selected_idx = None
    for line in gpt_reply.splitlines():
        if line.strip().startswith("선택"):
            import re
            m = re.search(r"\[(\d+)\]", line)
            if m:
                selected_idx = int(m.group(1))
                break

    # 파싱 실패 시 기본 1번 선택
    if selected_idx is None or selected_idx < 1 or selected_idx > len(docs):
        selected_idx = 1

    best_doc = docs[selected_idx - 1]
    return best_doc, gpt_reply


# ─── 3) 예시 검색 + GPT-4o 최종 선택 ───────────────────────────────────────────────
if __name__ == "__main__":
    # (가정) faiss_db는 이미 생성되어 로드된 FAISS 인덱스 객체입니다.
    # 예: faiss_db = FAISS.load_local(index_dir, OpenAIEmbeddings())

    # ① 조회할 사용자 질의
    query = "아기를 키우는 게 너무 힘들어요. 조언이 필요해요."

    # ② 기존 검색 함수 그대로 사용
    results = filtered_similarity_search(faiss_db, query, emotion="기쁨", relation="부모자녀")

    # ③ 검색 결과 출력
    for i, doc in enumerate(results, start=1):
        print(f"\n✅ 검색 결과 {i}")
        print(f"파일명: {doc.metadata['filename']}")
        print(f"감정: {doc.metadata['emotion']} / 관계: {doc.metadata['relation']}")
        print(doc.page_content[:300] + "...\n")

    # ④ 검색 결과가 있으면, GPT-4o로 "가장 적절한" 문서 선택
    if results:
        best_doc, explanation = choose_best_doc_with_gpt(query, results, model="gpt-4o-mini")

        print("\n\n=== GPT-4o가 선택한 최종 응답 ===")
        print(f"■ 선택된 파일명: {best_doc.metadata['filename']}")
        print(f"■ 선택 이유:\n{explanation}\n")
        print(f"■ 최종 응답 내용:\n{best_doc.page_content}")
    else:
        print("❗ 검색 결과가 없어 GPT 평가를 진행할 수 없습니다.")


In [ ]:
import os
import openai
import unicodedata
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# ─── 0) OpenAI API 키 설정 ─────────────────────────────────────────────────
openai.api_key = os.getenv("OPENAI_API_KEY")


# ─── 1) 사용자 검색 함수 그대로 ────────────────────────────────────────────────
def filtered_similarity_search(vectorstore, query, emotion=None, relation=None, k=3):
    all_docs = vectorstore.docstore._dict.values()
    filtered_docs = [
        doc for doc in all_docs
        if (emotion is None or doc.metadata.get("emotion") == emotion)
        and (relation is None or relation in doc.metadata.get("relation"))
    ]

    if not filtered_docs:
        print("❗ 해당 조건의 문서가 없습니다.")
        return []

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    query_chunks = splitter.split_text(query)

    search_results = []
    for chunk in query_chunks:
        search_results.extend(vectorstore.similarity_search(chunk, k=k))
    return search_results


# ─── 2) GPT-4o로 “최적 후보 선택 + 이유 설명” 함수 (v1 API) ───────────────────────────
def choose_best_doc_with_gpt(query, docs, model="gpt-4o-mini"):
    """
    query: 사용자의 원래 질문
    docs: filtered_similarity_search에서 반환된 Document 리스트
    model: "gpt-4o" 또는 "gpt-4o-mini"
    반환: (best_doc, gpt_reason)
      - best_doc: GPT가 선택한 Document 객체
      - gpt_reason: "선택: [번호]\n이유: ..." 형태의 문자열
    """
    prompt_parts = [
        "당신은 후보 응답을 평가하는 전문가입니다.\n",
        f"사용자 질문: \"{query}\"\n",
        "다음은 검색된 응답 후보들입니다.\n"
    ]

    for idx, doc in enumerate(docs, start=1):
        snippet = doc.page_content.strip().replace("\n", " ")
        if len(snippet) > 300:
            snippet = snippet[:300] + "..."
        prompt_parts.append(
            f"[{idx}]\n"
            f"Filename: {doc.metadata.get('filename')}\n"
            f"Emotion: {doc.metadata.get('emotion')}, Relation: {doc.metadata.get('relation')}\n"
            f"Content: \"{snippet}\"\n"
        )

    prompt_parts.append(
        "\n위 후보들 중에서, 사용자 질문에 가장 적절한 응답을 한 개만 선택하고, 그 이유를 간단히 설명해주세요.\n"
        "반드시 다음 형식으로 응답해 주세요:\n"
        "선택: [번호]\n"
        "이유: [간단한 설명]\n"
    )

    full_prompt = "\n".join(prompt_parts)

    # ── OpenAI Chat Completions 호출 ─────────────────────────────────────────────
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 뛰어난 대화 평가자입니다."},
            {"role": "user", "content": full_prompt}
        ],
        max_tokens=300,
        temperature=0.0
    )

    gpt_reply = response.choices[0].message.content.strip()

    # “선택: [번호]” 파싱
    selected_idx = None
    for line in gpt_reply.splitlines():
        if line.strip().startswith("선택"):
            import re
            m = re.search(r"\[(\d+)\]", line)
            if m:
                selected_idx = int(m.group(1))
                break

    # 파싱 실패 시 기본 1번
    if selected_idx is None or selected_idx < 1 or selected_idx > len(docs):
        selected_idx = 1

    best_doc = docs[selected_idx - 1]
    return best_doc, gpt_reply


# ─── 3) “선택된 후보를 간결하게 재작성” 함수 ─────────────────────────────────────
def generate_final_answer(query, best_doc, model="gpt-4o-mini"):
    """
    query: 사용자의 원래 질문
    best_doc: choose_best_doc_with_gpt가 반환한 Document 객체
    model: "gpt-4o" 또는 "gpt-4o-mini"
    반환: GPT가 생성한 최종 사용자용 응답(불필요한 부분 제거된 형태)
    """
    # (A) 프롬프트 구성: “최종 응답 후보”를 직접 재작성하도록 요청
    prompt = (
        "다음은 사용자의 질문과, 선택된 최적 응답 후보입니다.\n\n"
        f"사용자 질문: \"{query}\"\n"
        "선택된 후보 응답 내용(원문):\n"
        f"\"\"\"\n{best_doc.page_content}\n\"\"\"\n\n"
        "위 원문에서, 불필요한 반복/인사말/개인정보 등은 모두 제거하고, "
        "사용자가 이해하기 쉽도록 핵심만 남겨 간결하게 재작성해주세요.\n"
        "문체는 친절하고 공감 가득한 톤을 유지해 주시고, "
        "최종 답변만 출력해 주세요."
    )

    # (B) GPT-4o 호출
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 친절하고 공감능력이 뛰어난 상담사입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )

    final_answer = response.choices[0].message.content.strip()
    return final_answer


# ─── 4) 전체 흐름 예시 ────────────────────────────────────────────────────────
if __name__ == "__main__":
    # (가정) faiss_db는 이미 생성/로드된 FAISS 인덱스 객체입니다.
    # 예: faiss_db = FAISS.load_local(index_dir, OpenAIEmbeddings())

    # ① 사용자 질의
    query = "아기를 키우는 게 너무 힘들어요. 조언이 필요해요."

    # ② 기존 검색 함수 그대로 사용
    results = filtered_similarity_search(faiss_db, query, emotion="기쁨", relation="부모자녀")

    # ③ 검색 결과 출력 (원본 후보 3개)
    for i, doc in enumerate(results, start=1):
        print(f"\n✅ 검색 결과 {i}")
        print(f"파일명: {doc.metadata['filename']}")
        print(f"감정: {doc.metadata['emotion']} / 관계: {doc.metadata['relation']}")
        print(doc.page_content[:300] + "...\n")

    # ④ 검색 결과가 있으면, GPT-4o로 “가장 적절한” 문서 선택 + 이유 얻기
    if results:
        best_doc, explanation = choose_best_doc_with_gpt(query, results, model="gpt-4o-mini")
        print("\n\n=== GPT-4o가 선택한 최종 응답 후보 ===")
        print(f"■ 선택된 파일명: {best_doc.metadata['filename']}")
        print(f"■ 선택 이유:\n{explanation}\n")

        # ⑤ 선택된 후보를 재작성하여 최종 답변 생성
        cleaned_answer = generate_final_answer(query, best_doc, model="gpt-4o-mini")
        print("=== 최종 사용자 응답 (불필요한 내용 제거됨) ===")
        print(cleaned_answer)

    else:
        print("❗ 검색 결과가 없어 GPT 평가를 진행할 수 없습니다.")


In [ ]:
# ─── 예시: 여러 개의 query를 한 번에 처리해 보는 코드 ─────────────────────

# (가정) 이미 아래 함수들이 정의되어 있고, faiss_db도 생성/로드되어 있다고 봅니다.
# - filtered_similarity_search(vectorstore, query, emotion, relation)
# - choose_best_doc_with_gpt(query, docs, model)
# - generate_final_answer(query, best_doc, model)

# 1) 테스트할 질의 리스트를 정의
queries = [
    "아기를 키우는 일을 시작하려는데, 어떻게 준비해야 할까요?",
    "아이가 자꾸 밤에 깨서 낮잠도 잘 못 자요. 어떻게 도와줄 수 있을까요?",
    "육아 스트레스를 푸는 방법이 있을까요?",
    "첫 돌 지난 아기가 말을 잘 안 들을 때 어떻게 해야 하나요?",
    "아기가 갑자기 울음을 멈추지 않아서 당황스러워요. 조언 부탁드려요."
]

# 2) 감정(emotion)과 관계(relation) 예시는 고정해도 되고,
#    아니면 query별로 달리 지정해도 됩니다. 여기서는 예시로 전부 "기쁨"/"부모자녀"로 가정.
emotion = "기쁨"
relation = "부모자녀"

# 3) 각 query 순회하면서 단계별로 결과 출력
for idx, q in enumerate(queries, start=1):
    print(f"\n\n========== Query #{idx} ==========")
    print(f"사용자 질문: {q}\n")

    # 3-1) 감정/관계 필터 + FAISS 유사도 검색 → 후보 3개 가져오기
    candidates = filtered_similarity_search(faiss_db, q, emotion=emotion, relation=relation)
    if not candidates:
        print("❗ 조건에 맞는 문서가 없습니다. 다음 질의로 넘어갑니다.")
        continue

    # 3-2) 후보 원문 간단 출력
    print("■ 검색된 후보 (최대 3개):")
    for i, doc in enumerate(candidates, start=1):
        print(f"\n[후보 {i}] 파일명: {doc.metadata['filename']}")
        print(f"감정: {doc.metadata['emotion']}, 관계: {doc.metadata['relation']}")
        print(doc.page_content[:200].replace("\n", " ") + "...\n")

    # 3-3) GPT-4o에게 “가장 적절한 후보 선택 + 이유” 요청
    best_doc, choice_reason = choose_best_doc_with_gpt(q, candidates, model="gpt-4o-mini")
    print("\n■ GPT-4o가 선택한 후보:")
    print(f"  • 선택된 파일명: {best_doc.metadata['filename']}")
    print(f"  • 선택 이유:\n{choice_reason}\n")

    # 3-4) 선택된 후보를 다듬어서 최종 답변 생성
    final_answer = generate_final_answer(q, best_doc, model="gpt-4o-mini")
    print("■ 최종 사용자 응답 (정제된 텍스트):")
    print(final_answer)


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import os
import openai
import unicodedata
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# ─── 0) OpenAI API 키 설정 ─────────────────────────────────────────────────
openai.api_key = os.getenv("OPENAI_API_KEY")

# ─── 1) JSON 로드 + 메타데이터 추출 함수 ─────────────────────────────────────────
def load_documents_with_metadata(folder_path):
    documents = []
    for raw_filename in os.listdir(folder_path):
        filename = unicodedata.normalize("NFC", raw_filename)
        file_path = os.path.join(folder_path, raw_filename)

        if not os.path.isfile(file_path):
            continue
        if not filename.endswith(".json"):
            continue

        try:
            parts = filename.replace(".json", "").split("_")
            emotion = parts[1] if len(parts) > 1 else "unknown"
            relation = parts[2] if len(parts) > 2 else "unknown"

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                utterances = data.get("utterances", [])
                full_text = "\n".join([utt.get("text","") for utt in utterances])
                if full_text.strip() == "":
                    continue

                doc = Document(
                    page_content=full_text,
                    metadata={"filename": filename, "emotion": emotion, "relation": relation}
                )
                documents.append(doc)
        except Exception as e:
            print(f"❌ 오류 발생 ({filename}): {e}")

    return documents

# ─── 2) 문서 분할 함수 ─────────────────────────────────────────────────────
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    return splitter.split_documents(documents)

# ─── 3) FAISS 인덱스 생성 혹은 로드 함수 ───────────────────────────────────
def create_or_load_faiss(index_dir, split_docs):
    embeddings = OpenAIEmbeddings()
    if os.path.isdir(index_dir) and os.path.exists(os.path.join(index_dir, "index.faiss")):
        faiss_db = FAISS.load_local(index_dir, embeddings, allow_dangerous_deserialization=True)
        print("✅ 기존 FAISS 인덱스를 로드했습니다.")
    else:
        faiss_db = FAISS.from_documents(split_docs, embeddings)
        os.makedirs(index_dir, exist_ok=True)
        faiss_db.save_local(index_dir)
        print("✅ 새로운 FAISS 인덱스를 생성하고 저장했습니다.")
    return faiss_db

# ─── 4) 필터 + 유사도 검색 함수 ────────────────────────────────────────────────
def filtered_similarity_search(vectorstore, query, emotion=None, relation=None, k=3):
    all_docs = vectorstore.docstore._dict.values()
    filtered_docs = [
        doc for doc in all_docs
        if (emotion is None or doc.metadata.get("emotion") == emotion)
        and (relation is None or relation in doc.metadata.get("relation"))
    ]

    if not filtered_docs:
        return []

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    query_chunks = splitter.split_text(query)

    search_results = []
    for chunk in query_chunks:
        search_results.extend(vectorstore.similarity_search(chunk, k=k))
    return search_results

# ─── 5) 후보 중 최고 문서 선택 함수 ─────────────────────────────────────────────
def choose_best_doc_with_gpt(query, docs, model="gpt-4o-mini"):
    prompt_parts = [
        "당신은 대화 응답 후보를 평가하는 전문가입니다.\n",
        f"사용자 질문: \"{query}\"\n",
        "다음은 검색된 응답 후보들입니다.\n"
    ]

    for idx, doc in enumerate(docs, start=1):
        snippet = doc.page_content.strip().replace("\n", " ")
        if len(snippet) > 300:
            snippet = snippet[:300] + "..."
        prompt_parts.append(
            f"[{idx}]\n"
            f"Filename: {doc.metadata.get('filename')}\n"
            f"Emotion: {doc.metadata.get('emotion')}, Relation: {doc.metadata.get('relation')}\n"
            f"Content: \"{snippet}\"\n"
        )

    prompt_parts.append(
        "\n위 후보들 중에서, 사용자 질문에 가장 적절한 응답을 하나 선택하고, 그 이유를 간단히 설명해주세요.\n"
        "반드시 다음 형식으로 응답해 주세요:\n"
        "선택: [번호]\n"
        "이유: [간단한 설명]\n"
    )

    full_prompt = "\n".join(prompt_parts)

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 뛰어난 대화 평가자입니다."},
            {"role": "user", "content": full_prompt}
        ],
        max_tokens=300,
        temperature=0.0
    )

    gpt_reply = response.choices[0].message.content.strip()
    selected_idx = None
    for line in gpt_reply.splitlines():
        if line.strip().startswith("선택"):
            import re
            m = re.search(r"\[(\d+)\]", line)
            if m:
                selected_idx = int(m.group(1))
                break

    if selected_idx is None or selected_idx < 1 or selected_idx > len(docs):
        selected_idx = 1

    best_doc = docs[selected_idx - 1]
    return best_doc, gpt_reply

# ─── 6) 최종 답변 간결하게 생성 함수 ─────────────────────────────────────────────
def generate_final_answer(query, best_doc, model="gpt-4o-mini"):
    prompt = (
        "다음은 사용자의 질문과, 선택된 최적 응답 후보입니다.\n\n"
        f"사용자 질문: \"{query}\"\n"
        "선택된 후보 응답 내용(원문):\n"
        f"\"\"\"\n{best_doc.page_content}\n\"\"\"\n\n"
        "위 원문에서, 불필요한 반복/인사말/개인정보 등은 모두 제거하고, "
        "사용자가 이해하기 쉽도록 핵심만 남겨 간결하게 재작성해주세요.\n"
        "문체는 친절하고 공감 가득한 톤을 유지해 주시고, "
        "최종 답변만 출력해 주세요."
    )

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 친절하고 공감능력이 뛰어난 상담사입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )

    final_answer = response.choices[0].message.content.strip()
    return final_answer

# ─── 7) Gradio 응용: 채팅 인터페이스 구축 ───────────────────────────────────────
index_dir = "/content/drive/MyDrive/2025_Bigdata_nlp_class/faiss_index"
folder_path = "/content/drive/MyDrive/2025_Bigdata_nlp_class/aihub_dataset/Training/02_label_data"

# 문서 로드 및 FAISS 초기화
documents = load_documents_with_metadata(folder_path)
split_docs = split_documents(documents)
faiss_db = create_or_load_faiss(index_dir, split_docs)

def chat_response(query, emotion, relation):
    candidates = filtered_similarity_search(faiss_db, query, emotion, relation)
    if not candidates:
        return "조건에 맞는 문서가 없습니다."

    best_doc, _ = choose_best_doc_with_gpt(query, candidates, model="gpt-4o-mini")
    final_answer = generate_final_answer(query, best_doc, model="gpt-4o-mini")
    return final_answer

with gr.Blocks() as demo:
    gr.Markdown("## 감정/관계 기반 Empathy QA 시스템")
    with gr.Row():
        txt_query = gr.Textbox(label="질문", placeholder="질문을 입력하세요...", lines=2)
    with gr.Row():
        txt_emotion = gr.Textbox(label="Emotion (예: 기쁨, 당황, 분노)", placeholder="ex) 기쁨")
        txt_relation = gr.Textbox(label="Relation (예: 부모자녀, 부부, 연인)", placeholder="ex) 부모자녀")
    btn_submit = gr.Button("전송")
    output = gr.Textbox(label="답변", lines=5)

    btn_submit.click(chat_response, inputs=[txt_query, txt_emotion, txt_relation], outputs=output)

demo.launch()
